In [59]:
pip install imblearn

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

In [0]:
df = pd.read_csv('/content/file2.csv', low_memory = False)

In [62]:
df.head()

,Unnamed: 0,iday,extended,crit3,suicide,property,targtype1,attacktype1,gname,ransom,weaptype1,country_txt,country_txt.1,success
0,39,31,0,0,0,0,4,9,New People's Army (NPA),0.0,13,Philippines,Philippines,0
1,169,2,0,0,0,0,4,9,New People's Army (NPA),0.0,13,Philippines,Philippines,1
2,226,25,0,1,0,1,6,3,Palestinians,0.0,6,Turkey,Turkey,1
3,364,26,0,1,0,1,17,2,Irish Republican Army (IRA),0.0,8,United Kingdom,United Kingdom,0
4,365,27,0,1,0,0,14,2,Irish Republican Army (IRA),0.0,5,United Kingdom,United Kingdom,1


In [0]:
#embedding the gname 
le_gname = preprocessing.LabelEncoder()
le_country = preprocessing.LabelEncoder()
le_gname.fit(df['gname'])
le_country.fit(df['country_txt'])

array_g = np.array(le_gname.transform(df['gname']))
array_g = pd.DataFrame({'gname_embedded': array_g[:]})

array_c= np.array(le_country.transform(df['country_txt']))
array_c = pd.DataFrame({'country_embedded': array_c[:]})

In [0]:
df_embedded = df.assign(gname = array_g['gname_embedded'])
df_embedded = df_embedded.assign(country_txt = array_c['country_embedded'])

In [65]:
df_embedded

,Unnamed: 0,iday,extended,crit3,suicide,property,targtype1,attacktype1,gname,ransom,weaptype1,country_txt,country_txt.1,success
0,39,31,0,0,0,0,4,9,22,0.0,13,30,Philippines,0
1,169,2,0,0,0,0,4,9,22,0.0,13,30,Philippines,1
2,226,25,0,1,0,1,6,3,24,0.0,6,39,Turkey,1
3,364,26,0,1,0,1,17,2,13,0.0,8,41,United Kingdom,0
4,365,27,0,1,0,0,14,2,13,0.0,5,41,United Kingdom,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57673,181679,31,0,0,0,0,4,2,14,NaN,5,16,Iraq,0
57674,181680,31,0,0,0,-9,4,3,4,NaN,6,32,Somalia,1
57675,181683,31,0,1,0,0,17,2,29,NaN,5,0,Afghanistan,1
57676,181686,31,0,0,0,-9,4,2,4,NaN,5,32,Somalia,1


In [66]:
df_embedded = df_embedded.drop('Unnamed: 0', axis = 1)
df_embedded = df_embedded.drop('country_txt.1', axis = 1)
#df_embedded = df_embedded.drop('iyear', axis = 1)
df_embedded

,iday,extended,crit3,suicide,property,targtype1,attacktype1,gname,ransom,weaptype1,country_txt,success
0,31,0,0,0,0,4,9,22,0.0,13,30,0
1,2,0,0,0,0,4,9,22,0.0,13,30,1
2,25,0,1,0,1,6,3,24,0.0,6,39,1
3,26,0,1,0,1,17,2,13,0.0,8,41,0
4,27,0,1,0,0,14,2,13,0.0,5,41,1
...,...,...,...,...,...,...,...,...,...,...,...,...
57673,31,0,0,0,0,4,2,14,NaN,5,16,0
57674,31,0,0,0,-9,4,3,4,NaN,6,32,1
57675,31,0,1,0,0,17,2,29,NaN,5,0,1
57676,31,0,0,0,-9,4,2,4,NaN,5,32,1


In [0]:
labels = np.array(df_embedded['success'])
df_embedded = df_embedded.drop('success', axis = 1)
feature_list = list(df_embedded.columns)

In [68]:
#feature_list.remove('Unnamed: 0')
print(labels, len(feature_list))

[0 1 1 ... 1 1 1] 11


In [0]:
df_embedded = df_embedded.fillna(0)

In [70]:
df_embedded

,iday,extended,crit3,suicide,property,targtype1,attacktype1,gname,ransom,weaptype1,country_txt
0,31,0,0,0,0,4,9,22,0.0,13,30
1,2,0,0,0,0,4,9,22,0.0,13,30
2,25,0,1,0,1,6,3,24,0.0,6,39
3,26,0,1,0,1,17,2,13,0.0,8,41
4,27,0,1,0,0,14,2,13,0.0,5,41
...,...,...,...,...,...,...,...,...,...,...,...
57673,31,0,0,0,0,4,2,14,0.0,5,16
57674,31,0,0,0,-9,4,3,4,0.0,6,32
57675,31,0,1,0,0,17,2,29,0.0,5,0
57676,31,0,0,0,-9,4,2,4,0.0,5,32


In [0]:
df_embedded = np.array(df_embedded)

In [0]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [0]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=42, shuffle=False)

In [88]:
#train_features, test_features, train_labels, test_labels = train_test_split(df_embedded, labels, test_size = 0.25, random_state = 42)
accuracy = []
for train_index, test_index in cv.split(df_embedded):
    train_features, test_features, train_labels, test_labels = df_embedded[train_index], df_embedded[test_index], labels[train_index], labels[test_index]
    smt = SMOTE()
    train_features, train_labels = smt.fit_sample(train_features, train_labels)
    rf.fit(train_features, train_labels)
    smt_test = SMOTE()
    test_features, test_labels = smt_test.fit_sample(test_features, test_labels)
    y_pred=rf.predict(test_features)
    print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))
    accuracy.append(metrics.accuracy_score(test_labels, y_pred))
print(np.mean(accuracy))

Accuracy: 0.8832586786114222
Accuracy: 0.9095532146749001
Accuracy: 0.8482573229514275
Accuracy: 0.7857684290316468
Accuracy: 0.8885829662261381
Accuracy: 0.9003173417957813
Accuracy: 0.8852701702442635
Accuracy: 0.8603110599078341
Accuracy: 0.821584984358707
Accuracy: 0.8050721561969439
0.8587976323999064
